In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.transforms import Compose
from torch.utils.tensorboard import SummaryWriter

In [ ]:
class Discriminator(nn.Module):
  def __init__(self,img_dim):
    super().__init__()
    self.disc = nn.Sequential(
        nn.Linear(img_dim, 128),
        nn.LeakyReLU(0.1),
        nn.Linear(128,1),
        nn.Sigmoid()
    )

  def forward(self, x):
    return self.disc(x)

In [ ]:
class Generator(nn.Module):
  def __init__(self,z_dim, img_dim):
    super().__init__()
    self.gen = nn.Sequential(
        nn.Linear(z_dim,256),
        nn.LeakyReLU(0.1),
        nn.Linear(256, img_dim),
        nn.Tanh(),
    )

  def forward(self,x):
    return self.gen(x)

In [ ]:
##Hyperparameter
device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 3e-4
z_dim = 64 #128, 256
image_dim = 28*28*1
batch_size = 32
num_epochs = 50

In [ ]:
#initialisation
disc = Discriminator(image_dim).to(device)
gen = Generator(z_dim, image_dim).to(device)
fixed_noise = torch.randn((batch_size, z_dim)).to(device)
transforms = transforms.Compose(
      [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)

In [ ]:
#Data Loading 
dataset = datasets.MNIST(root="dataset/", transform = transforms , download=True)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
opt_disc = optim.Adam(disc.parameters(), lr=lr)
opt_gen = optim.Adam(gen.parameters(), lr=lr)
criterion = nn.BCELoss()
writer_fake = SummaryWriter(f"runs/GAN_MNIST/fake")
writer_real = SummaryWriter(f"runs/GAN_MNIST/real")
step = 0

In [ ]:
##Training
for epoch in range(num_epochs):
  for batch_idx, (real, _) in enumerate(loader):
    real = real.view(-1, 784).to(device)#Reshape, keep the number of examples intact and flatten everything else
    batch_size = real.shape[0]

    #Training Discriminator > max log(D(real)) + log(1-D(G(z)))
    noise = torch.randn(batch_size, z_dim).to(device)
    fake = gen(noise)
    disc_real = disc(real).view(-1)#D(real)
    lossD_real = criterion(disc_real, torch.ones_like(disc_real))
    disc_fake = disc(fake).view(-1)#D(G(z))
    lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
    lossD = (lossD_real + lossD_fake)/2
    disc.zero_grad()
    lossD.backward(retain_graph=True)
    opt_disc.step()

    #Training Generator min log(1-D(G(z))) ----- max log(D(G(z)))
    output = disc(fake).view(-1)
    lossG = criterion(output, torch.ones_like(output))
    gen.zero_grad()
    lossG.backward()
    opt_gen.step()


    #For Tensorboard
    if batch_idx == 0:
            print(
                f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(loader)} \
                      Loss D: {lossD:.4f}, loss G: {lossG:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
                data = real.reshape(-1, 1, 28, 28)
                img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
                img_grid_real = torchvision.utils.make_grid(data, normalize=True)

                writer_fake.add_image(
                    "Mnist Fake Images", img_grid_fake, global_step=step
                )
                writer_real.add_image(
                    "Mnist Real Images", img_grid_real, global_step=step
                )
                step += 1

Epoch [0/50] Batch 0/1875                       Loss D: 0.7582, loss G: 0.6445
Epoch [1/50] Batch 0/1875                       Loss D: 0.5062, loss G: 0.8890
Epoch [2/50] Batch 0/1875                       Loss D: 0.4243, loss G: 1.2975
Epoch [3/50] Batch 0/1875                       Loss D: 0.6946, loss G: 0.8387
Epoch [4/50] Batch 0/1875                       Loss D: 0.5115, loss G: 1.0136
Epoch [5/50] Batch 0/1875                       Loss D: 0.9317, loss G: 0.6590
Epoch [6/50] Batch 0/1875                       Loss D: 0.5054, loss G: 1.2146
Epoch [7/50] Batch 0/1875                       Loss D: 0.8856, loss G: 0.8944
Epoch [8/50] Batch 0/1875                       Loss D: 0.6224, loss G: 1.2709
Epoch [9/50] Batch 0/1875                       Loss D: 0.9650, loss G: 0.6419
Epoch [10/50] Batch 0/1875                       Loss D: 0.6525, loss G: 0.7992
Epoch [11/50] Batch 0/1875                       Loss D: 0.6287, loss G: 1.2207
Epoch [12/50] Batch 0/1875                       L